In [1]:
import pickle
import cPickle
from collections import defaultdict

In [2]:
items_set=pickle.load(open('../data/processed_data/item_set','rb')) 
item_data=pickle.load(open('../data/processed_data/all_items','rb'))
item_id_lookup = pickle.load(open('../data/processed_data/item_id_lookup','rb'))
item_name_map=pickle.load(open('../data/processed_data/item_name_map','rb'))

In [3]:
print item_data[0]

{'original_price': u'$14.99', 'discount_original_price': '', 'discount_final_price': '', 'tags': [u'Tower Defense', u'Strategy', u'Indie', u'Sci-fi', u'Co-op', u'Space', u'Multiplayer', u'Adventure', u'Singleplayer', u'Kickstarter', u'Local Co-Op'], 'reviews': [u'Mixed', u'Very Positive'], 'appid': '221540', 'date': u'Sep 23, 2014'}


In [4]:
item_appid_set = set()
for item in item_data:
    item_appid_set.add(int(item['appid']))
print len(item_appid_set)

#存储需要去掉的item index
item_index_to_remove = set()
for item_ind in items_set:
    if item_id_lookup[item_ind] not in item_appid_set:
        item_index_to_remove.add(item_ind)
print len(item_index_to_remove)

10258
304


In [5]:
# 建立 old 和 new index 之间的map
old_new_item_index_map = dict()
new_old_item_index_map = dict()
new_index = 0
for old_index in items_set:
    if old_index not in item_index_to_remove:
        old_new_item_index_map[old_index] = new_index
        new_old_item_index_map[new_index] = old_index
        new_index += 1
print len(old_new_item_index_map)

2515


In [6]:
#dict: user index --> item index set
user_item_map=pickle.load(open('../data/processed_data/user_item_map','rb'))

In [7]:
# temporary user item map,把每个user的item 的 index 都改为最新的。user index没有更新，在下一段处理。
temp_user_item_map = defaultdict(set)
for user_index in user_item_map:
    for old_item_index in user_item_map[user_index]:
        if old_item_index not in item_index_to_remove:
            new_item_index = old_new_item_index_map[old_item_index]
            temp_user_item_map[user_index].add(new_item_index)
print len(temp_user_item_map)

29634


In [8]:
# 发现上一段处理中没有去掉任何user
for u in temp_user_item_map:
    if len(temp_user_item_map[u]) <= 0:
        print u

In [9]:
#dict: bundle_index --> items set
bundle_item_map=pickle.load(open('../data/processed_data/bundle_item_map','rb')) 
print len(bundle_item_map)

615


In [10]:
# old bundle index 和 new bundle index 之间的map
old_new_bundle_index_map = dict()
new_old_bundle_index_map = dict()
# 新的 bundle item map
new_bundle_item_map = dict()
new_bundle_index = 0
for old_bundle_index in bundle_item_map:
    temp_set = set()
    for old_item_index in bundle_item_map[old_bundle_index]:
        if old_item_index not in item_index_to_remove:
            new_item_index = old_new_item_index_map[old_item_index]
            temp_set.add(new_item_index)
    if len(temp_set) > 0:
        new_bundle_item_map[new_bundle_index] = temp_set
        old_new_bundle_index_map[old_bundle_index] = new_bundle_index
        new_old_bundle_index_map[new_bundle_index] = old_bundle_index
        new_bundle_index += 1

print len(new_bundle_item_map)
cPickle.dump(new_bundle_item_map, open('../data/new_data/bundle_item_map.pkl', 'w'))

608


In [11]:
user_bundle_map=pickle.load(open('../data/processed_data/user_bundle_map','rb')) 

In [12]:
# 发现更新bundle 后，user 7171 没有任何bundle了，因此需要建立新的user list。
for u in user_bundle_map:
    bs = 0
    for b in user_bundle_map[u]:
        if b in old_new_bundle_index_map:
            bs += 1
    if bs <= 0:
        print u

7171


In [13]:
# temporary user bundle map
temp_user_bundle_map = defaultdict(set)
for u in user_bundle_map:
    for old_bundle_index in user_bundle_map[u]:
        if old_bundle_index in old_new_bundle_index_map:
            new_bundle_index = old_new_bundle_index_map[old_bundle_index]
            temp_user_bundle_map[u].add(new_bundle_index)
print len(temp_user_bundle_map)


29633


In [14]:
# 建立 old user index 和 new user index 之间的 map
old_new_user_index_map = dict()
new_old_user_index_map = dict()
# 新的 new_user_bundle_map
new_user_bundle_map = defaultdict(set)
new_user_index = 0
for old_user_index in user_bundle_map:
    if old_user_index in temp_user_bundle_map:
        old_new_user_index_map[old_user_index] = new_user_index
        new_old_user_index_map[new_user_index] = old_user_index
        new_user_bundle_map[new_user_index] = temp_user_bundle_map[old_user_index]
        new_user_index += 1
print len(new_user_bundle_map)
print len(old_new_user_index_map)
print old_new_user_index_map[7172]
cPickle.dump(new_user_bundle_map, open('../data/new_data/user_bundle_map.pkl', 'w'))

29633
29633
7171


In [15]:
# 新的 new_user_item_map
new_user_item_map = dict()
for old_user_index in temp_user_item_map:
    if old_user_index in old_new_user_index_map:
        new_user_index = old_new_user_index_map[old_user_index]
        new_user_item_map[new_user_index] = temp_user_item_map[old_user_index]
print len(new_user_item_map)
cPickle.dump(new_user_item_map, open('../data/new_data/user_item_map.pkl', 'w'))

29633
